# PCA

- PCA (Principle Component Analysis) finds directions of maximum variances
- The first principal component of the data is the direction in which the data varies the most


In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix

warnings.filterwarnings("ignore", category=FutureWarning)

# Read the data
seeds_df = pd.read_csv("datasets/Grains/seeds-width-vs-length.csv", header=None)
wiki_df = pd.read_csv("datasets/Wikipedia/wikipedia-vectors.csv", index_col=0)
print(seeds_df.shape)
seeds = seeds_df.values
#Sparse matrix
articles = csr_matrix(wiki_df.transpose())
titles = list(wiki_df.columns)
print(wiki_df.shape)
print(articles.shape)

## Pearson Correlation


In [ ]:
# Assign the 0th column of grains: width
width = seeds[:, 0]

# Assign the 1st column of grains: length
length = seeds[:, 1]

# Scatter plot width vs length
plt.scatter(width, length)
plt.axis("equal")
plt.show()

# Calculate the Pearson correlation
correlation, pvalue = pearsonr(width, length)

# Display the correlation
print(correlation)

## Decorrelate data using PCA


In [ ]:
# Create PCA instance: model
model = PCA()

# Apply the fit_transform method of model to grains: pca_features
pca_features = model.fit_transform(seeds)

# Assign 0th column of pca_features: xs
xs = pca_features[:, 0]

# Assign 1st column of pca_features: ys
ys = pca_features[:, 1]

# Scatter plot xs vs ys
plt.scatter(xs, ys)
plt.axis("equal")
plt.show()

# Calculate the Pearson correlation of xs and ys
correlation, pvalue = pearsonr(xs, ys)

# Display the correlation
print(correlation)

# Intrensic Dimension

- PCA identify Intrensic dimension
- Intrensic dimension = number of PCA features with significant variances


## Find the first principle component

- The first principal component of the data is the direction in which the data varies the most
- The principal components obtained from the fitted PCA model are printed. These are the eigenvectors that represent the directions of maximum variance in the dataset.


In [ ]:
# Make a scatter plot of the untransformed points
plt.scatter(seeds[:, 0], seeds[:, 1])

# Create a PCA instance: model
model = PCA()

# Fit model to points
model.fit(seeds)

# Get the mean of the grain samples: mean
mean = model.mean_

# Get the first principal component: first_pc
first_pc = model.components_[0, :]

# Plot first_pc as an arrow, starting at mean
plt.arrow(mean[0], mean[1], first_pc[0], first_pc[1], color="red", width=0.01)

# Keep axes on same scale
plt.axis("equal")
plt.show()

## Variance of the PCA features

- This Example has 1 intrensic dimension, the one with the largest variance


In [ ]:
# Create scaler: scaler
scaler = StandardScaler()

# Create a PCA instance: pca
pca = PCA()

# Create pipeline: pipeline
pipeline = make_pipeline(scaler, pca)

# Fit the pipeline to 'samples'
pipeline.fit(seeds)

# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_)
plt.xlabel("PCA feature")
plt.ylabel("variance")
plt.xticks(features)
plt.show()

# Dimension Reduction with PCA


In [ ]:
# Create a PCA model with 1 components: pca
pca = PCA(n_components=1)

# Fit the PCA instance to the scaled samples
pca.fit(seeds)

# Transform the scaled samples: pca_features
pca_features = pca.transform(seeds)

# Print the shape of pca_features
print(pca_features.shape)

## TruncatedSVD


In [ ]:
# Create a TruncatedSVD instance: svd
svd = TruncatedSVD(n_components=50)

# Create a KMeans instance: kmeans
kmeans = KMeans(n_clusters=6)

# Create a pipeline: pipeline
pipeline = make_pipeline(svd, kmeans)

# Fit the pipeline to articles
pipeline.fit(articles)

# Calculate the cluster labels: labels
labels = pipeline.predict(articles)

# Create a DataFrame aligning labels and titles: df
df = pd.DataFrame({"label": labels, "article": titles})

# Display df sorted by cluster label
print(df.sort_values("label"))